<a href="https://colab.research.google.com/github/mkdib1/University_PWs/blob/main/AudioSignal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import zipfile
import os
from google.colab import drive
from shutil import copyfile
import shutil
import tarfile
import IPython.display as ipd
from matplotlib import pyplot as plt
from scipy.io import wavfile as wav
from scipy.fft import rfft, fft, irfft, ifft, fftfreq
from scipy.signal import fftconvolve
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
drive.mount('/content/drive')
tar_path = '/content/drive/MyDrive/recordings.tar'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
with tarfile.open(tar_path, 'r') as tar_ref:
    tar_ref.extractall('/content/drive/MyDrive/audio_unzipped')

In [ ]:
# shutil.rmtree('/content/drive/MyDrive/audio_unzipped')

## Basic i/o

Vediamo come leggere un file .wav (audio)

In [ ]:
zero_jackson_rate, zero_jackson = wav.read("/content/drive/MyDrive/audio_unzipped/recordings/0_jackson_0.wav")

# wav.read() restituisce 2 parametri: il primo è la Freq di campionamento (scalare), il secondo è la traccia audio stessa

In [ ]:
zero_jackson_rate # isepzionando il primo parametro otteniamo lo scalare 8000. Significa che per ogni secondo di voce sono stati raccolti 8000 campioni

In [ ]:
zero_jackson

In [ ]:
# il secondo è codificato sotto forma di array numpy, e pertanto sarà una lista di numeri reali
# potremmo essere interessati a vederne la shape
zero_jackson.shape  # otteniamo che l'array contiene 5148 numeri. L'array in qst caso è monodimensionale. Solitamente le tracce audio sono codificati in shape N x 2 , dove 2
                    # indica il doppio canale orecchio dx/ orecchio sx


Riproduciamo ora la traccia

In [ ]:
ipd.Audio(zero_jackson, rate=zero_jackson_rate) # per farlo usiamo ipd.Audio, cui si passano come parametri la traccia stessa e
                                                # il sample rate

Proviamo ora a riprodurre la traccia con sample_rate  dimezzato

In [ ]:
ipd.Audio(zero_jackson, rate=zero_jackson_rate*0.5) # stiamo dicendo: "distribuisci 5148/2 campioni in un secondo di traccia audio"

La seguente riga di codice serve per scrivere la traccia audio, eventualmente manipolata.
Per farlo passiamo come parametri:
* il path
* sample_rate desiderato
* il nome del file originale

In [ ]:
# wav.write("/content/drive/MyDrive", rate=zero_jackson_rate, data=zero_jackson)

## Plot

In [ ]:
plt.plot(zero_jackson); plt.show() # qst è il grafico dell'audio, utilizzando come variabili il numero dei campioni (asse x)
                                  # e l'ampiezza del segnale (asse y)
                                  # Possiamo notare che in effetti lungo l'asse x i valori sono nel range [0,5000]

In [ ]:
# Volendo tuttavia apprezzare il dettaglio, chiediamo di stampare solo il range [0,500]
plt.plot(zero_jackson[0:500]); plt.show()

Come si può notare, l'onda ottenuta è **complessa**, nel senso che è composta da tante componenti che fanno sì che si sviluppi lungo valori diversi per ogni timestamp espressi in una distanza minima in termini di sample_rate (asse x)

Adesso proviamo con un'altra traccia

In [ ]:
one_jackson_rate, one_jackson = wav.read("/content/drive/MyDrive/audio_unzipped/recordings/1_jackson_1.wav")
ipd.Audio(one_jackson, rate=one_jackson_rate)

In [ ]:
plt.plot(one_jackson[0:500]); plt.show()

A livello di complessità, è uguale alla precedente più o meno. Possiamo notaer che i valori si sviluppano perlopiù in picchi discendenti verso valori bassi

# ANALISI DI FOURIER

Come visto a lezione, si tratta del cambio di **Dominio** di rappresentazione di un segnale, il segnale audio, da quello del tempo a quello delle frequenze.

Sebbene lungo l'asse x non ci sia il tempo ma la sequenza di campioni, possiamo cmq approssimarla ad una linea temporale di un campione dopo l'altro.

Con l'applicazione della trasformata di Fourier, lungo l'asse x avrò le frequenze di ciascuna componente del segnale.

In [ ]:
# Applicazione della trasformata di Fourier
zero_jackson_fft = fft(zero_jackson) # passiamo come parametro la traccia salvata

In [ ]:
zero_jackson_fft.shape

Studiandone la forma, vediamo come la fft non ha alterato la complessità del segnale giacchè la shape è rimasta uguale all'originale (abbiamo sempre 5148 componenti)

In [ ]:
plt.plot(zero_jackson_fft)

Notiamo come la forma sia completamente diversa dalla traccia originale.
Tuttavia ci manca un passaggio

In [ ]:
# Salviamoci un parametro per il plot, che sarebbero le "giuste label" da dare all'asse x.
# Notiamo come nel plot precedente, lungo l'asse x abbiamo ancora la sequenza dei campioni e non le frequenze

zero_jackson_freqs = fftfreq(len(zero_jackson), 1.0/zero_jackson_rate) # 'fftfreq' restituisce un vettore che useremo come label dell'asse x, costruita con la
                                                                      # lunghezza della traccia, e l'inverso del sample_rate, cioè quanti secondi sono allocati per ciascun campione

plt.plot(zero_jackson_freqs, zero_jackson_fft)  # la misura appena calcolata la si può usare come parametro aggiuntivo del plot


Adesso abbiamo ottenuto che lungo l'asse x ci sono le frequenze, e quindi il plot ha rispecchiato il nostro dato. Nello specifico, sono state identificate le basse frequenze al centro.

D'ora in poi però, ci serviremo della func **rfft** per convertire i dati sul dominio delle frequenze.
Sarebbe una sorta di scorciatoia.

In [ ]:
zero_jackson_rfft = rfft(zero_jackson)
plt.plot(zero_jackson_rfft)

Come si nota, il grafico è esattamente la metà del grafico precedente, con sole frequenze positive (è stato eliminato il semi-asse negativo di x).

## Ricostruzione dell'onda trasformata con Fourier

Come da teoria, sappiamo che l'analisi di Fourier serve a decomporre l'onda originale che risulta complessa, in componenti più semplici.
Ad esempio potremmo dire che il grafico precedente è stato ottenuto dalla sovrapposizione di un primo tracciato sinusoidale ad una certa frequenza d'onda, cui si aggiunge in sovrapposizione un tracciato più semplice, poi un'altra sinusoide complessa e così via (vedi quaderno).

Andiamo pertanto a plottare il grafico in zoom differenti per osservare le componenti dell'onda.

In [ ]:
plt.plot(zero_jackson_rfft[0:100]); plt.show()

In [ ]:
plt.plot(zero_jackson_rfft[200:300]); plt.show()

In [ ]:
plt.plot(zero_jackson_rfft[1000:1050]); plt.show()

## Low pass / High pass filtering

In [ ]:
# Ripartiamo sempre dalla traccia mappata nel dominio delle frequenze.
# Quello che vogliamo fare è eliminare ad es. le basse frequenze dal segnale
# originale.
# Ristampiamo quindi il segnale
plt.plot(zero_jackson_rfft)

# Di questo segnale vogliamo "salvare" solo le basse frequenze, ovvero,
# dato che siamo nel dominio delle frequenze per cui in asse x abbiamo le
# freqs ordinate, il tutto di traduce nel voler "salvare" i primi n-valori.
# In effetti notiamo che a partire da n>500, si ottengono datapoints trascurabili

Costruiamo quindi il filtro. Lo facciamo manualmente.
Per farlo, istanzieremo un vettore numpy della stessa len() di len(segnale)
il quale avrà valore == 1 fino ad un certo n, e poi avrà valori == 0.

In [ ]:
low_pass_filter = np.zeros(len(zero_jackson_rfft))
low_pass_filter[:500] = 1
plt.plot(low_pass_filter)

Quella plottata è la forma del nostro filtro.
Adesso usiamolo, mediante prodotto vettoriale con il nostro segnale

In [ ]:
zero_jackson_filtered = zero_jackson_rfft * low_pass_filter
plt.plot(zero_jackson_filtered)

Abbiamo annullato tutte le freqs > 500.

## From FREQS to TIME

Adesso, prendiamo il segnale filtrato e ritorniamo nel dominio del tempo, e riascoltiamo il nuovo segnale.
In altre parole, bisogna applicare la **trasformata inversa**.

In [ ]:
# Usiamo 'irfft'
zero_jackson_low_filtered_time = irfft(zero_jackson_filtered)
ipd.Audio(zero_jackson_low_filtered_time, rate=zero_jackson_rate)


In [ ]:
# Confrontiamolo con il segnale originale
ipd.Audio(zero_jackson, rate=zero_jackson_rate)

Si nota una compressione dei dati (abbiamo annulato il 70% dei dati).
Il suono sembra "ovattato".
L'applicazione della FFT è proprio la compressione del dato, per alleggerire la memoria.
La cosa interessante è che l'info,sotto certe condizioni, rimane preservata, nel senso che si riesce ancora a percepire che si sta pronunciando la parola "zero".
Se noi avessimo compresso l'info con un filtro troncando ai primi 500 valori dell'asse-x nel dominio del tempo, stavamo applicando un filtro temporale e pertanto l'effetto sarebbe stato di ascoltare solo le prime lettere del segnale audio.

In [ ]:
# Usiamo ora un filtro passa alto
high_pass_filter = np.zeros(len(zero_jackson_rfft))
high_pass_filter[500:] = 1
plt.plot(high_pass_filter)

In [ ]:
# Applichiamo il filtro al segnale
zero_jackson_filtered = zero_jackson_rfft * high_pass_filter
plt.plot(zero_jackson_filtered)

In [ ]:
# Ritorniamo al dominio del tempo e ascoltiamo il nuovo segnale audio
zero_jackson_high_filtered_time = irfft(zero_jackson_filtered)
ipd.Audio(zero_jackson_high_filtered_time, rate=zero_jackson_rate)

Stessa cosa di prima, il suono di sente in modo più acuto ma preserva l'informazione acustica.

# CONVOLUTION

Tratteremo il tema non in termini di deep NN, ma come strumento di applicazione di un filtro o kernel (matrice piu piccola) al segnale originale.
Matematicamete si tratta di una moltiplicazione tra matrici, di cui una più piccola dell'altra.
Il kernel "scorrerà" sul segnale.

In [ ]:
# Importiamo una nuova traccia
five_nicolas_rate, five_nicolas_21 = wav.read("/content/drive/MyDrive/audio_unzipped/recordings/5_nicolas_21.wav")

In [ ]:
# Ascoltiamo il segnale audio
ipd.Audio(five_nicolas_21, rate=five_nicolas_rate)

In [ ]:
# !pip install pydub
from pydub import AudioSegment

# # # Reading .mp3 audio signals
# hello = AudioSegment.from_mp3("/content/drive/MyDrive/hello.mp3")
# bang = AudioSegment.from_mp3("/content/drive/MyDrive/bang.mp3")

# # # Converting into .wav
# hello.export("/content/drive/MyDrive/hello.wav", format="wav")
# bang.export("/content/drive/MyDrive/bang.wav", format="wav")

hello_rate, hello = wav.read("/content/drive/MyDrive/hello.wav")

In [ ]:
# ipd.Audio(hello, rate=hello_rate)
print(type(hello))  # Dovrebbe essere <class 'numpy.ndarray'> o una lista di numeri
print(hello.shape)


In [ ]:
import IPython.display as ipd

# Riproduci il file audio con il sample rate corretto
ipd.Audio(hello, rate=hello_rate)